In [ ]:
import os
import tempfile
import unittest
import filecmp
import sbol3
import paml
import tyto
import uml
import json

from paml_convert.autoprotocol.autoprotocol_specialization import AutoprotocolSpecialization
from paml_convert.autoprotocol.strateos_api import StrateosAPI, StrateosConfig
from autoprotocol import container_type as ctype
from paml.execution_engine import ExecutionEngine
from container_api.client_api import matching_containers, strateos_id

%load_ext autoreload
%autoreload 2

In [ ]:
## Execute a PAML protocol, emitting an Autoprotocol protocol

# Setup PAML protocol for execution
out_dir = "."
doc = sbol3.Document()
sbol3.set_namespace('https://bbn.com/scratch/')
protocol_file = os.path.join(os.getcwd(), "../test/testfiles", "igem_ludox_test.nt")
doc.read(protocol_file, 'nt')
protocol = doc.find("https://bbn.com/scratch/iGEM_LUDOX_OD_calibration_2018")
agent = sbol3.Agent("test_agent")


# Setup Output location and Credentials for Strateos
out_dir = "out"
if not os.path.exists(out_dir):
    os.mkdir(out_dir)
autoprotocol_output = os.path.join(out_dir, "test_LUDOX_autoprotocol.json")
secrets_file = os.path.join(os.getcwd(), "../secrets/strateos_secrets.json")


# Setup Autoprotocol specialization
resolutions = {
    doc.find("https://bbn.com/scratch/LUDOX") : "rs1b6z2vgatkq7",
    doc.find("https://bbn.com/scratch/ddH2O") : "rs1c7pg8qs22dt",
    "container_id" : "ct1g9qsg4wx6gcj"
}
api = StrateosAPI(cfg=StrateosConfig.from_file(secrets_file))
autoprotocol_specialization = AutoprotocolSpecialization(autoprotocol_output, api, resolutions)


# Execute the protocol
ee = ExecutionEngine(specializations=[autoprotocol_specialization])
parameter_values = [paml.ParameterValue(parameter=protocol.get_input("wavelength"), 
                                        value=uml.LiteralIdentified(value=sbol3.Measure(100, tyto.OM.nanometer)))]
execution = ee.execute(protocol, agent, id="test_execution", parameter_values=parameter_values)

# Return the instructions from the protocol
ee.specializations[0].protocol.instructions

In [ ]:
# Display the Autoprotocol JSON
with open(autoprotocol_output, "r") as f:
    autoprotocol_json = json.loads(f.read())
autoprotocol_json

In [ ]:
# Submit Autoprotocol object for execution at Strateos

st = api.get_strateos_connection()
response = st.analyze_run(ee.specializations[0].protocol, test_mode=True)
response

In [ ]:
#response = st.submit_run(ee.specializations[0].protocol, 
#                         project_id=api.cfg.project_id,
#                         title=protocol.name,
#                         test_mode=True)

#response